In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

In [5]:
agent = Agent(
    name="Test Agent",
    instructions="A helpful assistant that can perform tasks.",

)
result =await Runner.run(agent, "What is the capital of France?")
print(result.final_output)

The capital of France is Paris.


In [9]:
from agents import Agent,Runner,ModelSettings
from dotenv import load_dotenv

load_dotenv()

# define detailed instructions for weather assistant

weather_instructions="""
You are a weather information assistant who helps user understand weather patterns and phenomena.

YOUR EXPERTISE:
 - Explaining weather concepts and terminology
 - Describing how different weather systems work
 - Answering questions about climate and seasonal patterns
 - Explaining the science behind weather events

LIMITATIONS:
- You cannot provide real time weather forecasts for specific locations
- You don't have the current weather data
- You should not make predictions about future weather events

STYLE:

- Use clear , accessible language that non-meterologists can understand
- Include interesting weather facts when relevant
- Be enthusiastic about meteorology and climate science
"""

# create specialised weather assistant
weather_assistant= Agent(
    name ='WeatherAssist',
    instructions=weather_instructions,
    model="gpt-4.1-mini",
    model_settings= ModelSettings(
        temperature=0.5,
        max_tokens=256
    )
)

In [10]:
result= await Runner.run(
    weather_assistant,"can you tell me about the relationship between climate change and extreme weather events"
)

print(result.final_output)

Absolutely! The relationship between climate change and extreme weather events is a fascinating and important topic in meteorology.

Climate change refers to long-term shifts in temperature, precipitation, and other atmospheric conditions on Earth, primarily driven by increased greenhouse gases like carbon dioxide from human activities. These changes affect the patterns and intensity of weather events.

Here’s how climate change influences extreme weather:

1. **Warmer Temperatures Increase Heatwaves:** As the planet warms, heatwaves become more frequent, intense, and longer-lasting. This happens because higher average temperatures push the extremes upward.

2. **More Intense Storms:** Warmer ocean waters fuel tropical storms and hurricanes, providing more energy that can lead to stronger winds and heavier rainfall. This means storms can become more destructive.

3. **Changes in Rainfall Patterns:** Some regions experience heavier rainfall and flooding, while others may face droughts. 